In [39]:
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

import sys
from subsp_scripts import tax2table

import statsmodels.api as sm

from collections import defaultdict
import random

from upsetplot import UpSet

In [21]:
# Read sample metadata from CureatedMetagenomicData

metadata = pd.read_csv("../machine_learning/data/2023_curatedmetadata.csv", low_memory=False)
metadata = metadata.loc[metadata['study_condition'] == 'control']
metadata = metadata.loc[metadata['body_site'] == 'stool']
metadata = metadata.loc[metadata['age_category'] == 'adult']

In [22]:
# Prepare Tax table

humgut = pd.read_csv("../benchmark/HumGut.tsv",sep='\t')
humgut['cluster95'] = humgut['cluster95'].astype(str).str.rjust(4,'0')
humgut = humgut[['cluster95','gtdbtk_taxonomy']]

Tax = tax2table(humgut['gtdbtk_taxonomy'])
Tax = Tax.merge(humgut[['cluster95']],left_index=True,right_index=True)

In [23]:
# Read subspecies info

subsp_def = pd.read_csv('../benchmark/fullname_subsp_def_sourmash.txt',sep='\t')
subsp_def['subspecies'] = subsp_def['subspecies'].astype(str).str.rjust(10,'0')
subsp_def['cluster95'] = subsp_def['subspecies'].str[:4]
subsp_def = subsp_def[['subspecies','cluster95']].drop_duplicates()

In [24]:
# Keep only metadata for which we have samples

metadata = metadata.loc[metadata.country.isin(metadata['country'].value_counts().loc[metadata['country'].value_counts() >= 30].index)]

In [25]:
# Read mastiff prevalence table

mastiff = pd.read_csv("mastiff_output_prevalence.csv",index_col=0)

mastiff.columns = mastiff.columns.str.split(".",expand=True).get_level_values(0)
mastiff.columns = mastiff.columns.str.rjust(10,'0')

In [26]:
mastiff = mastiff.loc[mastiff.index.isin(metadata['NCBI_accession'])].T

In [27]:
mastiff = mastiff.T

In [28]:
metadata = metadata.set_index("NCBI_accession")[['country']]

In [29]:
country_continent_mapping = {"USA":"North America", "NLD":"Europe", "GBR":"Europe", "ISR":"Asia","DNK":"Europe", "CHN":"Asia", "DEU":"Europe", "ITA":"Europe",
                            "SWE":"Europe", "JPN":"Asia", "KAZ": "Asia", "IND":"Asia", "FJI": "Australia and Oceania", "CMR": "Africa", "FIN": "Europe",
                            "MNG":"Asia", "MDG": "Africa", "IRL": "Europe", "TZA":"Africa", "ESP":"Europe", "PER": "South America", "CAN":"North America",
                            "SLV": "Europe", "FRA":"Europe", "KOR":"Asia", "LUX":"Europe"}

In [30]:
metadata['continent'] = metadata.country.map(country_continent_mapping)

In [31]:
mastiff = mastiff.replace(True, 1)
mastiff = mastiff.replace(False, 0)

In [32]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from statsmodels.stats.multitest import multipletests
from statsmodels.sandbox.stats.multicomp import MultiComparison

from tqdm import tqdm

def chi_square_test(df, variable = 'subspecies'):
    # Get a list of subspecies
    subspecies_list = df[variable].unique()
    
    pvals = []
    # Loop over each subspecies
    for subspecies in tqdm(subspecies_list):
        subsp_df = df[df[variable] == subspecies].reset_index().drop_duplicates().set_index('index')
        # Create a contingency table
        contingency_table = pd.crosstab(subsp_df[subsp_df[variable] == subspecies]['country'], 
                                        subsp_df[subsp_df[variable] == subspecies]['presence'])

        # Perform the chi-square test
        chi2, p, dof, ex = chi2_contingency(contingency_table, correction=False)
        pvals.append(p)

    # Correct for multiple testing (BH method)
    reject, pvals_corrected, _, _ = multipletests(pvals, method='fdr_bh')

    result_list = []
    for subspecies, pval, reject in zip(subspecies_list, pvals_corrected, reject):
        print(f'Variable: {variable}, corrected p-value: {pval}')

        if reject:
            print(f'Significant result for {variable}. Performing post-hoc test...')

            # Subset df for significant subspecies and perform post-hoc test
            df_sub = df[df[variable] == subspecies]
            mc = MultiComparison(df_sub['presence'], df_sub['country'])
            result = mc.tukeyhsd()
            data_rows = result.summary().data
            subsp_results = pd.DataFrame(data_rows[1:],columns = data_rows[0])
            subsp_results[variable] = subspecies
            result_list.append(subsp_results)
    return pd.concat(result_list)

In [33]:
prevalence_df = mastiff.reset_index().melt(id_vars='sample')
prevalence_df = prevalence_df.merge(metadata,left_on = 'sample',right_index=True).reset_index(drop=True)
prevalence_df = prevalence_df.rename(columns={'variable':'subspecies', 'value':'presence'})

In [31]:
#result = chi_square_test(prevalence_df)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3038/3038 [36:19<00:00,  1.39it/s]


Variable: subspecies, corrected p-value: 1.0727583358380715e-152
Significant result for subspecies. Performing post-hoc test...


/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/scipy/integrate/_quadpack_py.py:1225: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


Variable: subspecies, corrected p-value: 6.564567930946106e-06
Significant result for subspecies. Performing post-hoc test...


/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/scipy/integrate/_quadpack_py.py:1225: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


...

/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/scipy/integrate/_quadpack_py.py:1225: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


Variable: subspecies, corrected p-value: 1.5482181699193975e-13
Significant result for subspecies. Performing post-hoc test...


/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/scipy/integrate/_quadpack_py.py:1225: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


Variable: subspecies, corrected p-value: 1.0
Variable: subspecies, corrected p-value: 2.6737461871876663e-55
Significant result for subspecies. Performing post-hoc test...



KeyboardInterrupt



In [36]:
result = pd.read_csv("countries_comparison.tsv",sep='\t')
result.subspecies = result.subspecies.astype(str).str.rjust(10,'0')